
  1.  Load heart disease dataset in pandas dataframe
  2.  Remove outliers using Z score. Usual guideline is to remove anything that has Z score > 3 formula or Z score < -3
  3.  Convert text columns to numbers using label encoding and one hot encoding
  4.  Apply scaling
  5.  Build a classification model using various methods (SVM, logistic regression, random forest) and check which model gives you the best accuracy
  6.  Now use PCA to reduce dimensions, retrain your model and see what impact it has on your model in terms of accuracy. Keep in mind that many times doing PCA reduces the accuracy but computation is much lighter and that's the trade off you need to consider while building models in real life


# Step 0: Insert libraries and Dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sn

# Step 1: Load heart disease dataset in pandas dataframe

In [2]:
df = pd.read_csv('/kaggle/input/pca-exercise/heart.csv')
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [3]:
df.shape

(918, 12)

In [4]:
df.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


# Step 2: Remove outliers

In [5]:
# Step 2: Remove outliers with z-score -3 to 3 in columns RestingBP, Cholesterol, FastingBS, MaxHR, 
# and Oldpeak

In [6]:
df.shape

(918, 12)

In [7]:
df1 = df[df.RestingBP>(df.RestingBP.mean()+3*df.RestingBP.std())]
df1 = df[df.RestingBP<(df.RestingBP.mean()+3*df.RestingBP.std())]

In [8]:
df2 = df1[df1.Cholesterol>(df1.Cholesterol.mean()+3*df1.Cholesterol.std())]
df2 = df1[df1.Cholesterol<(df1.Cholesterol.mean()+3*df1.Cholesterol.std())]

In [9]:
df3 = df2[df2.FastingBS>(df2.FastingBS.mean()+3*df2.FastingBS.std())]
df3 = df2[df2.FastingBS<(df2.FastingBS.mean()+3*df2.FastingBS.std())]

In [10]:
df4 = df3[df.MaxHR>(df3.MaxHR.mean()+3*df3.MaxHR.std())]
df4 = df3[df.MaxHR<(df3.MaxHR.mean()+3*df3.MaxHR.std())]

/tmp/ipykernel_42/2676693088.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df4 = df3[df.MaxHR>(df3.MaxHR.mean()+3*df3.MaxHR.std())]
/tmp/ipykernel_42/2676693088.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df4 = df3[df.MaxHR<(df3.MaxHR.mean()+3*df3.MaxHR.std())]


In [11]:
df5 = df4[df.Oldpeak>(df4.Oldpeak.mean()+3*df4.Oldpeak.std())]
df5 = df4[df.Oldpeak<(df4.Oldpeak.mean()+3*df4.Oldpeak.std())]

/tmp/ipykernel_42/2739043989.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df5 = df4[df.Oldpeak>(df4.Oldpeak.mean()+3*df4.Oldpeak.std())]
/tmp/ipykernel_42/2739043989.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df5 = df4[df.Oldpeak<(df4.Oldpeak.mean()+3*df4.Oldpeak.std())]


In [12]:
df.shape

(918, 12)

In [13]:
df5.shape

(902, 12)

**So we cleared 918 - 902 = 16 outliers!**

# Step 3: Convert text columns to numbers

In [14]:
df5.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [15]:
df5.Sex.unique()

array(['M', 'F'], dtype=object)

In [16]:
df5.ChestPainType.unique()

array(['ATA', 'NAP', 'ASY', 'TA'], dtype=object)

In [17]:
df5.RestingECG.unique()

array(['Normal', 'ST', 'LVH'], dtype=object)

In [18]:
df5.ExerciseAngina.unique()

array(['N', 'Y'], dtype=object)

In [19]:
df5.ST_Slope.unique()

array(['Up', 'Flat', 'Down'], dtype=object)

In [20]:
df6 = df5.copy()

df6.ChestPainType.replace(
    {
        'ATA': 0,
        'NAP': 1,
        'ASY': 2,
        'TA' : 3
    },
    inplace=True
)

df6.RestingECG.replace(
    {
        'Normal': 1,
        'ST': 2,
        'LVH': 3
    },
    inplace=True
)

df6.ExerciseAngina.replace(
    {
        'N': 0,
        'Y': 1
    },
    inplace=True
)

df6.ST_Slope.replace(
    {
        'Down': 1,
        'Flat': 2,
        'Up': 3
    },
    inplace=True
)

df6.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,0,140,289,0,1,172,0,0.0,3,0
1,49,F,1,160,180,0,1,156,0,1.0,2,1
2,37,M,0,130,283,0,2,98,0,0.0,3,0
3,48,F,2,138,214,0,1,108,1,1.5,2,1
4,54,M,1,150,195,0,1,122,0,0.0,3,0


In [23]:
df7 = pd.get_dummies(df6, dtype='int', drop_first=True)
df7.head()

,Age,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease,Sex_M
0,40,0,140,289,0,1,172,0,0.0,3,0,1
1,49,1,160,180,0,1,156,0,1.0,2,1,0
2,37,0,130,283,0,2,98,0,0.0,3,0,1
3,48,2,138,214,0,1,108,1,1.5,2,1,0
4,54,1,150,195,0,1,122,0,0.0,3,0,1


# Step 4: Apply Scaling

In [25]:
X = df7.drop('HeartDisease', axis='columns')
y = df7.HeartDisease

In [26]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled

array([[-1.42896269, -1.6990351 ,  0.46089071, ..., -0.84676261,
         1.0456339 ,  0.51485643],
       [-0.47545956, -0.52558207,  1.5925728 , ...,  0.14079864,
        -0.62072967, -1.94228905],
       [-1.74679706, -1.6990351 , -0.10495034, ..., -0.84676261,
         1.0456339 ,  0.51485643],
       ...,
       [ 0.37209878,  0.64787097, -0.10495034, ...,  0.33831089,
        -0.62072967,  0.51485643],
       [ 0.37209878, -1.6990351 , -0.10495034, ..., -0.84676261,
        -0.62072967, -1.94228905],
       [-1.64085227, -0.52558207,  0.3477225 , ..., -0.84676261,
         1.0456339 ,  0.51485643]])

# Step 5: Build a classification model using various methods (SVM, logistic regression, random forest) and check which model gives you the best accuracy

In [42]:
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [34]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=30)

In [35]:
model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    }    
}

In [36]:
from sklearn.model_selection import GridSearchCV

In [37]:
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(X_train, y_train)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df_model = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df_model

,model,best_score,best_params
0,svm,0.865441,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.851571,{'n_estimators': 10}
2,logistic_regression,0.850201,{'C': 1}


**The best score is from SVM (0.86)!**

# Step 6: Use PCA to reduce dimension

In [38]:
from sklearn.decomposition import PCA

pca = PCA(0.95)
X_pca = pca.fit_transform(X)
X_pca.shape

(902, 2)

In [39]:
pca.n_components_

2

In [45]:
X_train_pca, X_test_pca, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=30)

model = svm.SVC(gamma='auto')
model.fit(X_train_pca, y_train)
model.score(X_test_pca, y_test)

0.56353591160221

**With PCA we see that we lose accuracy of the model!**